In [ ]:
!pip install pypdf
!pip install langchain
!pip install google-generativeai
!pip install langchain_google_genai
!pip install chromadb

  Using cached pypdf-4.2.0-py3-none-any.whl (290 kB)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/RAG/paper.pdf")
pages = loader.load_and_split()
pages

In [ ]:
page = "".join([p.page_content for p in pages])

In [ ]:
import google.generativeai as genai
f = open("/content/drive/MyDrive/RAG/.geminiai_api_key.txt")
key = f.read()

genai.configure(api_key=key)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(pages)

print(len(chunks))

print(type(chunks[0]))

110
<class 'langchain_core.documents.base.Document'>


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
embedding_model= GoogleGenerativeAIEmbeddings(google_api_key=key, model="models/embedding-001")

In [ ]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")
db.persist()

In [ ]:
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [ ]:
retriever = db_connection.as_retriever(search_kwargs={"k":5})
print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

chat = model.start_chat(history=[])

chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-pro-latest',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction=None,
    ),
    history=[]
)

In [ ]:
user_input = page+ " previous text is the content of the research paper Leave No Context Behind .based on that answer the question" + "What is the most important topic that covered in the research paper named Leave No Context Behind"

response = chat.send_message(user_input)

print(response.text)

## Leave No Context Behind: Spotlight on Infini-attention

Analyzing the provided abstract and introduction, the research paper "Leave No Context Behind" centers on **Infini-attention**, a new attention mechanism designed to equip Transformer-based Large Language Models (LLMs) with the ability to handle infinitely long inputs while maintaining efficiency in memory usage and computation. 

**Why Infini-attention takes the lead:**

* **Tackling LLM Limitations:** Traditional LLMs face constraints due to the quadratic complexity of attention mechanisms, limiting their context window. Infini-attention introduces a compressive memory component that enables efficient processing of extended sequences without excessive memory demands.
* **Merging Local and Global Contexts:**  The paper underscores the significance of both local and global contexts in language comprehension. Infini-attention ingeniously combines masked local attention with long-term linear attention within a single Transformer 